<a href="https://colab.research.google.com/github/punkmic/pytorch_neural_network/blob/master/fine_tunning_covolution_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
try:
  import torch
  from torch import nn, optim
  import torch.nn.functional as F
  import torchvision
  from torchvision import datasets, transforms, models
except:
  !pip install torch

In [20]:
# Function to train the neural network on the image dataset
def train(model, train_loader, validation_loader, criterion, optimizer, device):
    """
    Trains a neural network on a given image dataset

    Parameters
    ----------
    model : nn.Module
        The neural network to be trained
    train_loader : DataLoader
        The training data in the form of PyTorch DataLoader
    validation_loader : DataLoader
        The validation data in the form of PyTorch DataLoader
    criterion : nn.Module
        The loss function used for training
    optimizer : torch.optim
        The optimizer used for training
    device : torch.device
        The device to run the training on

    Returns
    -------
    nn.Module
        The trained neural network
    """
    # Define number of training epochs
    epochs = 2
    # Set best loss value to an arbitrary high value
    best_loss = 1e6
    # Dictionary to store the train and validation data loaders
    image_dataset = {'train': train_loader, 'valid': validation_loader}
    # Counter to keep track of number of times validation loss increases
    loss_counter = 0

    # Loop through the epochs
    for epoch in range(epochs):
        # Loop through train and validation phases
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            # Set the model to train mode for train phase and eval mode for validation phase
            if phase == 'train':
                model.train()
            else:
                model.eval()
            # Initialize variables to keep track of running loss and accuracy
            running_loss = 0.0
            running_corrects = 0
            running_samples = 0

            # Loop through the data in the data loader
            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                # Transfer inputs and labels to the device
                inputs = inputs.to(device)
                labels = labels.to(device)
                # Get model outputs
                outputs = model(inputs)
                # Calculate the loss
                loss = criterion(outputs, labels)

                # Perform backpropagation and optimization only in train phase
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                # Get the predicted class from the model outputs
                _, preds = torch.max(outputs, 1)
                # Update running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                running_samples += len(inputs)
                # Print the current loss and accuracy every 2000 samples
                if running_samples % 2000 == 0:
                    accuracy = running_corrects/running_samples
                    # Printing the loss and accuracy details every 2000 iterations
                    print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%)".format(
                            running_samples,
                            len(image_dataset[phase].dataset),
                            100.0 * (running_samples / len(image_dataset[phase].dataset)),
                            loss.item(),
                            running_corrects,
                            running_samples,
                            100.0*accuracy,
                        )
                    )

            # NOTE: Uncomment the lines below to train and test on the whole dataset
            if running_samples>(0.1*len(image_dataset[phase].dataset)):
                break

            # Calculating the mean loss and accuracy of the epoch
            epoch_loss = running_loss / running_samples
            epoch_acc = running_corrects / running_samples

            # Saving the best validation loss and checking the loss increase counter
            if phase=='valid':
              if epoch_loss<best_loss:
                best_loss=epoch_loss
              else:
                loss_counter+=1

        # Breaking the epoch loop if the loss increase counter reaches 1
        if loss_counter==1:
          break
    
    return model

In [21]:
def test(model, test_loader, criterion):
    """
    Test the given model on the test dataset using the test_loader and criterion.
    
    Parameters:
    - model (nn.Module): the model to be tested
    - test_loader (DataLoader): the test dataset loader
    - criterion (nn.Module): the loss function to be used for testing
    
    Returns:
    - total_loss (float): the average loss on the test dataset
    - total_acc (float): the average accuracy on the test dataset
    """
    # set the model to evaluation mode
    model.eval() 

    # running loss for each batch
    running_loss = 0 
    
    # running number of correct predictions for each batch
    running_corrects = 0 

    # loop through the test_loader
    for inputs, labels in test_loader:
        outputs = model(inputs) # forward pass through the model
        loss = criterion(outputs, labels) # compute the loss
        _, preds = torch.max(outputs, 1) # get the predictions by finding the max value in the output
        running_loss += loss.item() * inputs.size(0) # accumulate the loss
        running_corrects += torch.sum(preds == labels.data).item() # accumulate the number of correct predictions

    # calculate the average loss and accuracy on the entire test dataset
    total_loss = running_loss / len(test_loader)
    total_acc = running_corrects / len(test_loader)
    
    return total_loss, total_acc


In [22]:
def create_model():
    """
    Creates a ResNet18 model with a modified fully connected layer
    
    Returns:
        The ResNet18 model with a modified fully connected layer
    """
    model = models.resnet18(pretrained=True)

    # Freeze all pre-trained parameters
    for param in model.parameters():
        param.requires_grad = False   

    # Get the number of features from the original fully connected layer
    num_features=model.fc.in_features
    
    # Replace the original fully connected layer with a new layer with 10 output units
    model.fc = nn.Sequential(
                   nn.Linear(num_features, 10))
    return model


In [ ]:
# Set batch size for training and testing datasets
batch_size = 10

# Define the transformations for training set
training_transform = transforms.Compose([
    # Randomly flip the image horizontally with a probability of 0.5
    transforms.RandomHorizontalFlip(p=0.5),
    # Resize the image to 224x224
    transforms.Resize(224),
    # Convert the image to a Tensor
    transforms.ToTensor(),
    # Normalize the image using the mean and standard deviation for the 3 color channels
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the transformations for testing set
testing_transform = transforms.Compose([
    # Convert the image to a Tensor
    transforms.ToTensor(),
    # Randomly resize and crop the image to 224x224
    transforms.RandomResizedCrop(224),
    # Normalize the image using the mean and standard deviation for the 3 color channels
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training set using the defined transformations
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

# Load the training data into a data loader for batch processing
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

# Load the testing set using the defined transformations
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

# Load the testing data into a data loader for batch processing
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)

# Create a model
model = create_model()

# Check if GPU is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on Device {device}")

# Move the model to the specified device
model = model.to(device)

# Define the criterion (loss function)
criterion = nn.CrossEntropyLoss()

# Define the optimizer for updating the model parameters
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Train the model using the trainloader, testloader, criterion, and optimizer
train(model, train_loader, test_loader, criterion, optimizer, device)

# Test the trained model using the test loader and criterion
test(model, test_loader, criterion)
